In [ ]:
import os
import shutil
from teemof.initialize import *
from teemof.run import initialize_trial

## Select sample files to modify
Different samples are available for interpenetrated and single MOF with different flux directions

Check *teemof/initialize.py* for available files

All available file paths are given in *sample_files* dictionary

In [ ]:
sample_dir = r'/home/kutay/Documents/git/tee_mof/sample'
sample_files = dict()
sample_files['inp'] = os.path.join(sample_dir, 'in3_ipmof.cond.sample')
sample_files['qsub'] = os.path.join(sample_dir, 'lammps_qsub.sh.sample')
sample_files['data'] = os.path.join(sample_dir, 'lammps_ipmof.data.sample')

inp_lines, data_lines, qsub_lines = get_files(sample_files)

## Trial directory

In [ ]:
trial_name = 'Trial7'
'
trial_dir = os.path.join(os.getcwd(), 'Trials', trial_name)
os.makedirs(trial_dir) if not os.path.exists(trial_dir) else None

## Single Trial
Here a single trial with multiple runs are initialized

In [ ]:
coefficient_list = [[1, 1, 0, 5], [2, 2, 0, 5], [1, 2, 0.1, 5]]
thermo_step = 100
masses = [[1, 39.948], [2, 20.1797]]

inp_lines = change_thermo(inp_lines, thermo=thermo_step)
inp_lines = change_pair_coeff(inp_lines, coefficient_list)
data_lines = change_masses(data_lines, masses)
qsub_lines = lammps_qsub(qsub_lines, name='TrialX', walltime='12:00:00', nodes=1, ppn=4, queue='shared')

number_of_runs = 10
initialize_trial(trial_dir, number_of_runs, inp_lines, data_lines, qsub_lines)

## Multiple Trials
Here a list of epsilon values are given to create different trials with given number of runs each 

In [ ]:
epsilon = [0.01, 0.05, 0.1, 0.5, 1, 5, 10]
sg = 4.75 
number_of_runs = 5
qsub_lines = lammps_qsub(qsub_lines, name=trial_name, walltime='06:00:00', nodes=1, ppn=4, queue='shared')

for trial_index, eps in enumerate(epsilon):
    new_trial_dir = os.path.join(trial_dir, '%s-%i' % (trial_name, trial_index + 1))  # Initialize trial directory
    if not os.path.exists(new_trial_dir):
        os.mkdir(new_trial_dir)
        
    coefficient_list = [[1, 1, 0, sg],
                        [2, 2, 0, sg],
                        [1, 2, eps, sg]]

    input_lines = change_thermo(inp_lines, thermo=100)
    input_lines = change_pair_coeff(inp_lines, coefficient_list)
    data_lines = change_masses(data_lines, [[1, 39.948], [2, mass]])
    
    run_info = dict(sigma=sg, epsilon=eps, name="Epsilon: %.2f" % eps, thermo=100, files=sample_files)
    initialize_trial(new_trial_dir, number_of_runs, input_lines, data_lines, qsub_lines, info=run_info)